# Problem Statement:

- Predict the burnt area of forest using Neural Networks

# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder 

import warnings
warnings.filterwarnings('ignore')

In [29]:
import keras
from tensorflow.keras.layers import Dense,  Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from tensorflow.keras.optimizers import Adam

Importing dataset

In [30]:
fire = pd.read_csv('forestfires.csv')

EDA

In [31]:
fire.head()

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,daymon,daysat,daysun,daythu,daytue,daywed,monthapr,monthaug,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,small


In [32]:
fire.shape

(517, 31)

In [33]:
fire['size_category'].unique()

array(['small', 'large'], dtype=object)

In [51]:
fire['area']

0       0.00
1       0.00
2       0.00
3       0.00
4       0.00
       ...  
512     6.44
513    54.29
514    11.16
515     0.00
516     0.00
Name: area, Length: 517, dtype: float64

In [34]:
fire1 = fire.copy()

Preprocessing

In [35]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()

fire1['month'] = lb.fit_transform(fire1['month'])
fire1['day'] = lb.fit_transform(fire1['day'])

In [37]:
fire1['size_category'] = lb.fit_transform(fire1['size_category'])

In [38]:
fire1.head()

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,daymon,daysat,daysun,daythu,daytue,daywed,monthapr,monthaug,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,7,0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,10,5,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,10,2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
3,7,0,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
4,7,3,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1


In [71]:
scalar = StandardScaler()

X_std = scalar.fit_transform(fire1[['month', 'day', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'size_category']])

In [72]:
X_std

array([[ 0.28422225, -1.42312073, -0.80595947, ...,  1.49861442,
        -0.07326831,  0.60640322],
       [ 0.97087134,  1.17671466, -0.00810203, ..., -1.74175564,
        -0.07326831,  0.60640322],
       [ 0.97087134, -0.38318657, -0.00810203, ..., -1.51828184,
        -0.07326831,  0.60640322],
       ...,
       [-1.08907592,  0.13678051, -1.64008316, ...,  1.49861442,
        -0.07326831, -1.64906775],
       [-1.08907592, -0.38318657,  0.68095666, ..., -0.00983371,
        -0.07326831,  0.60640322],
       [ 0.74198831,  1.17671466, -2.02087875, ...,  0.26950853,
        -0.07326831,  0.60640322]])

In [76]:
y = scalar.fit_transform(fire1[['area']])
y = y.reshape(len(y),1)[:,0]
y

array([-2.02019788e-01, -2.02019788e-01, -2.02019788e-01, -2.02019788e-01,
       -2.02019788e-01, -2.02019788e-01, -2.02019788e-01, -2.02019788e-01,
       -2.02019788e-01, -2.02019788e-01, -2.02019788e-01, -2.02019788e-01,
       -2.02019788e-01, -2.02019788e-01, -2.02019788e-01, -2.02019788e-01,
       -2.02019788e-01, -2.02019788e-01, -2.02019788e-01, -2.02019788e-01,
       -2.02019788e-01, -2.02019788e-01, -2.02019788e-01, -2.02019788e-01,
       -2.02019788e-01, -2.02019788e-01, -2.02019788e-01, -2.02019788e-01,
       -2.02019788e-01, -2.02019788e-01, -2.02019788e-01, -2.02019788e-01,
       -2.02019788e-01, -2.02019788e-01, -2.02019788e-01, -2.02019788e-01,
       -2.02019788e-01, -2.02019788e-01, -2.02019788e-01, -2.02019788e-01,
       -2.02019788e-01, -2.02019788e-01, -2.02019788e-01, -2.02019788e-01,
       -2.02019788e-01, -2.02019788e-01, -2.02019788e-01, -2.02019788e-01,
       -2.02019788e-01, -2.02019788e-01, -2.02019788e-01, -2.02019788e-01,
       -2.02019788e-01, -

In [81]:
type(X_std)

numpy.ndarray

In [80]:
type(y)

numpy.ndarray

Train-Test-Spli

In [82]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size = 0.2, random_state=42)

In [83]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((413, 11), (104, 11), (413,), (104,))

Model Building and Hyperparameter Tuning

In [91]:
def create_model(learning_rate,dropout_rate,activation_function,init,neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1,input_dim = 11,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neuron2,input_dim = neuron1,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation = 'relu'))
    
    adam = Adam(lr = learning_rate)
    model.compile(loss = 'mean_squared_error',optimizer = adam,metrics = ['mean_squared_error'])
    return model

In [92]:
# Creating model

model = KerasRegressor(build_fn=create_model, verbose=0)

In [97]:
# Defining GridSearch parametrs

batch_size = [20, 40]   
epochs = [50, 100]     
learning_rate = [0.001, 0.01]   
dropout_rate = [0.1, 0.2]
activation_function = ['relu', 'linear']    
init = ['uniform', 'normal']    
neuron1 = [4, 8, 16]
neuron2 = [2, 4, 8]

In [98]:
param_grids = dict(batch_size=batch_size, epochs=epochs, learning_rate=learning_rate, dropout_rate=dropout_rate,
                   activation_function=activation_function, init=init, neuron1=neuron1, neuron2=neuron2)

In [101]:
# Building gridSearch Model

grid = GridSearchCV(estimator=model, param_grid=param_grids, cv=KFold(), verbose=10, refit=True)

In [102]:
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 576 candidates, totalling 2880 fits
[CV] activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=50, init=uniform, learning_rate=0.001, neuron1=4, neuron2=2 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=50, init=uniform, learning_rate=0.001, neuron1=4, neuron2=2, score=-0.404, total=   1.3s
[CV] activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=50, init=uniform, learning_rate=0.001, neuron1=4, neuron2=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s


[CV]  activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=50, init=uniform, learning_rate=0.001, neuron1=4, neuron2=2, score=-0.062, total=   2.2s
[CV] activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=50, init=uniform, learning_rate=0.001, neuron1=4, neuron2=2 


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   12.9s remaining:    0.0s


[CV]  activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=50, init=uniform, learning_rate=0.001, neuron1=4, neuron2=4, score=-1.788, total=   1.8s
[CV] activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=50, init=uniform, learning_rate=0.001, neuron1=4, neuron2=4 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   14.7s remaining:    0.0s


Streaming output truncated to the last 5000 lines.
[CV] activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=50, init=uniform, learning_rate=0.001, neuron1=8, neuron2=4 
[CV]  activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=50, init=uniform, learning_rate=0.001, neuron1=8, neuron2=4, score=-0.091, total=   1.9s
[CV] activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=50, init=uniform, learning_rate=0.001, neuron1=8, neuron2=4 
[CV]  activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=50, init=uniform, learning_rate=0.001, neuron1=8, neuron2=4, score=-0.151, total=   1.6s
[CV] activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=50, init=uniform, learning_rate=0.001, neuron1=8, neuron2=4 
[CV]  activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=50, init=uniform, learning_rate=0.001, neuron1=8, neuron2=4, score=-1.788, total=   1.9s
[CV] activation_function=relu, batch_size=20, dropout_rate=0.2, epochs

[Parallel(n_jobs=1)]: Done 2880 out of 2880 | elapsed: 85.4min finished


Model Evaluation

In [103]:
grid_result.best_score_

-0.4602481722831726

In [104]:
grid_result.best_params_

{'activation_function': 'relu',
 'batch_size': 20,
 'dropout_rate': 0.2,
 'epochs': 100,
 'init': 'normal',
 'learning_rate': 0.001,
 'neuron1': 8,
 'neuron2': 8}

In [106]:
#Predict values based on new parameters
y_pred_acc = grid.predict(X_test)
y_pred_acc

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.2759222 , 0.12392834,
       0.12727557, 0.        , 0.        , 0.07673028, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.05235419, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.04953039, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.45615846, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.46379912, 0.        ,
       0.        , 0.23889028, 0.        , 0.        , 0.     

In [112]:
grid_result.score

<bound method BaseSearchCV.score of GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             error_score=nan,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7f961ce20910>,
             iid='deprecated', n_jobs=None,
             param_grid={'activation_function': ['relu', 'linear'],
                         'batch_size': [20, 40], 'dropout_rate': [0.1, 0.2],
                         'epochs': [50, 100], 'init': ['uniform', 'normal'],
                         'learning_rate': [0.001, 0.01], 'neuron1': [4, 8, 16],
                         'neuron2': [2, 4, 8]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=10)>

In [115]:
grid.score(X_test, y_test)

-2.7830276489257812

### Negative score:

- The actual MAE is simply the positive version of the number we’re getting.

- The unified scoring API always maximizes the score, so scores which need to be minimized are negated in order for the unified scoring API to work correctly. The score that is returned is therefore negated when it is a score that should be minimized and left positive if it is a score that should be maximized.